<!-- # Processing the final dataset
 -->


In [2]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder

In [3]:
def keep_rare_lu_types(unified_data):
    # Define the list of bands to check
    bands_to_check = ['lulc_sum_20', 'lulc_sum_21', 'lulc_sum_35', 
                    'lulc_sum_39', 'lulc_sum_40', 'lulc_sum_41', 
                    'lulc_sum_46', 'lulc_sum_48', 'lulc_sum_9']

    # Create a mask where at least one of the specified bands is non-zero
    mask = unified_data.select(bands_to_check).reduce(ee.Reducer.anyNonZero())

    # # Apply the mask to the unified_data
    return unified_data.updateMask(mask)



In [43]:
method = "mapbiomas"
year = 0
aggregate_LU = True

secondary = ee.Image(f"{data_folder}/{method}/secondary")
# nearest_mature = ee.Image(f"{data_folder}/{method}/nearest_mature_biomass_image").rename("nearest_mature_biomass")
distance_to_nearest_mature = ee.Image(f"{data_folder}/{method}/distance_to_nearest_mature")
cwd = ee.Image(f"{data_folder}/raw/cwd_chave").int16()
sur_cover = ee.Image(f"{data_folder}/{method}/sur_cover")
categorical = ee.Image(f"{data_folder}/categorical").select(["biome", "ecoreg", "protec", "indig"])
topography = ee.Image("CSP/ERGo/1_0/Global/ALOS_landforms").rename("topography")
soil = ee.Image(f"{data_folder}/soilgrids").select(['nitro', 'sand', 'phh2o'])  # Dropping multicollinear bands
climate = ee.Image(f"{data_folder}/yearly_terraclim")

distance_to_border_mask = ee.Image(f"{data_folder}/distance_to_border_mask").byte()
one_hectare_mask = ee.Image(f"{data_folder}/{method}/one_hectare_mask").selfMask()

base_name = "projects/amazon-forest-regrowth/assets/mapbiomas/nearest_mature_biomass_"
suffixes = range(1, 22)
images = [ee.Image(f"{base_name}{suffix}") for suffix in suffixes]
image_collection = ee.ImageCollection(images)
merged_image = image_collection.mosaic()
nearest_mature = merged_image.reproject(scale = 1000, crs = 'EPSG:4326').rename("nearest_mature_biomass")

# Combine all bands with additional masking conditions
combined_mask = nearest_mature.mask().And(distance_to_border_mask).And(one_hectare_mask)

unified_data = secondary.addBands([
    nearest_mature, sur_cover, cwd, distance_to_nearest_mature, 
    categorical, soil, topography, climate
])

if method == "mapbiomas":
    fire = ee.Image(f"{data_folder}/{method}/ESA_fires")
    unified_data = unified_data.addBands([fire])

    # if (aggregate_LU):
    #     suffix = "aggregated"
    # else:
    #     suffix = "non_aggregated"

    # if (year != 0):
    #     suffix += f"_{year}yr"
    # else:
    #     suffix += "_all"

    # land_use = ee.Image(f"{data_folder}/{method}/land_use_{suffix}")
    # unified_data = unified_data.addBands([land_use])
    # combined_mask = combined_mask.And(land_use.select("last_LU").mask())
else:
    suffix = "eu"
#     fire = ee.Image(f"{data_folder}/{method}/num_fires")
#     unified_data = unified_data.addBands([fire])


unified_data = unified_data.updateMask(combined_mask)
mask = unified_data.select("num_fires_before_regrowth").gt(0).selfMask()

# map = geemap.Map()
# map.addLayer(unified_data, {}, "Unified Data")
# map.addLayer(mask, {'min':0, 'max':1, 'palette':["red"]}, "Mask")
# map

In [47]:

# Function to preprocess images and create the unified data image
def create_unified_data(method, year):
    secondary = ee.Image(f"{data_folder}/{method}/secondary")
    # nearest_mature = ee.Image(f"{data_folder}/{method}/nearest_mature_biomass_image").rename("nearest_mature_biomass")
    distance_to_nearest_mature = ee.Image(f"{data_folder}/{method}/distance_to_nearest_mature")
    cwd = ee.Image(f"{data_folder}/raw/cwd_chave").int16()
    sur_cover = ee.Image(f"{data_folder}/{method}/sur_cover")
    categorical = ee.Image(f"{data_folder}/categorical").select(["biome", "ecoreg", "protec", "indig"])
    topography = ee.Image("CSP/ERGo/1_0/Global/ALOS_landforms").rename("topography")
    soil = ee.Image(f"{data_folder}/soilgrids").select(['nitro', 'sand', 'phh2o'])  # Dropping multicollinear bands
    climate = ee.Image(f"{data_folder}/yearly_terraclim")

    distance_to_border_mask = ee.Image(f"{data_folder}/distance_to_border_mask").byte()
    one_hectare_mask = ee.Image(f"{data_folder}/{method}/one_hectare_mask").selfMask()

    base_name = f"{data_folder}/{method}/nearest_mature_biomass_"
    suffixes = range(1, 22)
    images = [ee.Image(f"{base_name}{suffix}") for suffix in suffixes]
    image_collection = ee.ImageCollection(images)
    nearest_mature = image_collection.mosaic().rename("nearest_mature_biomass")
    if method == "mapbiomas":
        nearest_mature = nearest_mature.reproject(scale = 1000, crs = 'EPSG:4326').rename("nearest_mature_biomass")

    # Combine all bands with additional masking conditions
    combined_mask = nearest_mature.mask().And(distance_to_border_mask).And(one_hectare_mask)

    unified_data = secondary.addBands([
        nearest_mature, sur_cover, cwd, distance_to_nearest_mature, 
        categorical, soil, topography, climate
    ])

    if method == "mapbiomas":
        # fire = ee.Image(f"{data_folder}/{method}/ESA_fires")
        # unified_data = unified_data.addBands(fire.rename("ESA"))
        # suffix = "ESA_fire"
        
        if (aggregate_LU):
            suffix = "aggregated"
        else:
            suffix = "non_aggregated"

        if (year != 0):
            suffix += f"_{year}yr"
        else:
            suffix += "_all"

        land_use = ee.Image(f"{data_folder}/{method}/land_use_{suffix}")
        unified_data = unified_data.addBands([land_use])
        combined_mask = combined_mask.And(land_use.select("last_LU").mask())
    else:
        suffix = "eu"
    #     fire = ee.Image(f"{data_folder}/{method}/num_fires")
    #     unified_data = unified_data.addBands([fire])

    unified_data = unified_data.updateMask(combined_mask)

    return unified_data, suffix

# Main Function to run tile-wise exports
def export_tilewise(method, year):
    # Generate unified data image with selected layers
    unified_data, suffix = create_unified_data(method, year)
    # Filter properties to export
    to_remove = ['.geo', 'system:index']
    all_properties = unified_data.bandNames().getInfo()
    properties_to_export = [p for p in all_properties if p not in to_remove]

    # Load region of interest (ROI) and create a grid over the ROI
    grid = roi.coveringGrid("EPSG:4326", 500000)
    tile_ids = grid.aggregate_array('system:index').getInfo()
    count = 0

    # Loop over IDs
    for feature_id in tile_ids:
        count = count + 1
        feat = grid.filter(ee.Filter.eq('system:index', feature_id))

        unified_data_sampled = unified_data.sample(region = feat.geometry(), geometries = False)

        # Export task to Google Drive
        task = ee.batch.Export.table.toDrive(
            collection=unified_data_sampled,
            description=f'{suffix}_{count}',
            fileFormat='CSV',
            selectors=properties_to_export
        )
        task.start()
        print(f'Started export task {count}')

# Main Function to run tile-wise exports
def export_stratified(method, year):
    # Generate unified data image with selected layers
    unified_data, suffix = create_unified_data(method, year)
    # Filter properties to export
    to_remove = ['.geo', 'system:index']
    all_properties = unified_data.bandNames().getInfo()
    properties_to_export = [p for p in all_properties if p not in to_remove]

    unified_data_sampled = unified_data.stratifiedSample(numPoints = 15000, classBand = 'biome', geometries = False)

    # Export task to Google Drive
    task = ee.batch.Export.table.toDrive(
        collection=unified_data_sampled,
        description=f'{suffix}',
        fileFormat='CSV',
        selectors=properties_to_export
    )
    task.start()


# aggregate_LU = True
# export_stratified('eu', 0)
# export_stratified('mapbiomas', 0)
# export_stratified('mapbiomas', 5)
# export_stratified('mapbiomas', 10)
# export_stratified('mapbiomas', 15)
# aggregate_LU = False
# export_stratified('mapbiomas', 0)
# export_stratified('mapbiomas', 5)
# export_stratified('mapbiomas', 10)
# export_stratified('mapbiomas', 15)

aggregate_LU = True
export_tilewise('eu', 0)
export_tilewise('mapbiomas', 0)
export_tilewise('mapbiomas', 5)
export_tilewise('mapbiomas', 10)
export_tilewise('mapbiomas', 15)
# aggregate_LU = False
# export_tilewise('mapbiomas', 0)
# export_tilewise('mapbiomas', 5)
# export_tilewise('mapbiomas', 10)
# export_tilewise('mapbiomas', 15)

EEException: User memory limit exceeded.

In [8]:
secondary_mapbiomas = ee.Image(f"{data_folder}/mapbiomas/secondary")
secondary_eu = ee.Image(f"{data_folder}/eu/secondary")

secondary_mapbiomas = secondary_mapbiomas.updateMask(secondary_eu.select("age"))
secondary_eu = secondary_eu.updateMask(secondary_mapbiomas.select("age"))
secondary_mapbiomas = secondary_mapbiomas.updateMask(secondary_eu.select("age"))

categorical = ee.Image(f"{data_folder}/categorical")
biome_mask = categorical.select('biome').eq(1) \
               .Or(categorical.select('biome').eq(4)) \
               .Or(categorical.select('biome').eq(6))
categorical = categorical.updateMask(biome_mask)
cwd = ee.Image(f"{data_folder}/raw/cwd_chave")

unified_data = secondary_mapbiomas.addBands([categorical, cwd, \
                                             secondary_mapbiomas.select("age").rename("age_eu")])\
                                                .updateMask(secondary_mapbiomas.select("age"))

unified_data_sampled = unified_data.stratifiedSample(
    numPoints = 10000, classBand = 'biome', region = roi
)

task = ee.batch.Export.table.toDrive(
    collection = unified_data_sampled, description = "mapbiomas_eu", fileFormat = 'CSV'
)
task.start()

In [ ]:
# to get the data from current agricultural land, to predict its regrowth potential:

# get all land use history for all plots that are currently agricultural land
# get the nearest mature biomass for each of these plots
# get the same predictors and run the model for these pixels.